In [5]:
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import time
import re
import itertools
import sys
import pickle
import os
import string
import autoreload

from random import shuffle
from IPython.display import clear_output, display, Image, HTML
from scipy.spatial.distance import cosine

sys.path.append('../neural_programmer')

from notebook_utils import init_data, build_graph, restore_model
from neural_programmer import evaluate

In [6]:
%reload_ext autoreload
%autoreload 2

## Paths, parameters, etc. 

In [7]:
# Use only one GPU on the multi-GPU machine
os.environ["CUDA_VISIBLE_DEVICES"]="1"

# WikiTableQuestions data
DATA_DIR = 'wtq_data'
PERTURBED_DATA_DIR = 'perturbed_wtq_data'

# Pretrained model
MODEL_FILE = os.path.join('pretrained_model', 'model_92500')

# Output directory to write attributions
OUT_DIR = '/tmp'

## Load data, build graph and restore pretrained weights

In [9]:
train_data, dev_data, test_data, utility = init_data(DATA_DIR)

Annotated examples loaded  14152
Annotated examples loaded  4344
entry match token:  9133 9133
entry match token:  9134 9134
# train examples  10178
# dev examples  2546
# test examples  3913


In [12]:
tf.reset_default_graph()
sess, graph, params = build_graph(utility)

forget gate bias
step:  0
step:  1
step:  2
step:  3
optimize params  ['unit', 'word', 'word_match_feature_column_name', 'controller', 'column_controller', 'column_controller_prev', 'controller_prev', 'question_lstm_ix', 'question_lstm_fx', 'question_lstm_cx', 'question_lstm_ox', 'question_lstm_im', 'question_lstm_fm', 'question_lstm_cm', 'question_lstm_om', 'question_lstm_i', 'question_lstm_f', 'question_lstm_c', 'question_lstm_o', 'history_recurrent', 'history_recurrent_bias', 'break_conditional']
grads:  Tensor("gradients/L2Loss_grad/mul:0", shape=(15, 256), dtype=float64) unit
grads:  Tensor("gradients/L2Loss_1_grad/mul:0", shape=(10800, 256), dtype=float64) word
grads:  Tensor("gradients/L2Loss_2_grad/mul:0", shape=(1,), dtype=float64) word_match_feature_column_name
grads:  Tensor("gradients/L2Loss_3_grad/mul:0", shape=(512, 256), dtype=float64) controller
grads:  Tensor("gradients/L2Loss_4_grad/mul:0", shape=(512, 256), dtype=float64) column_controller
grads:  Tensor("gradients/L

In [13]:
sess, graph = restore_model(sess, graph, params, MODEL_FILE)

INFO:tensorflow:Restoring parameters from /home/pramodkm/interpreting_np/trained_model/model_92500


In [14]:
evaluate(sess, dev_data, utility.FLAGS.batch_size, graph, 92500)

dev set accuracy   after  92500  :  0.3720472446811481
2540.0 2546
--------


## Apply Integrated Gradients (IG) 